In [1]:
import numpy as np
import open3d as o3d
import rosbag
from sensor_msgs.msg import PointCloud2
import sensor_msgs.point_cloud2 as pc2
from nav_msgs.msg import Path
import pandas as pd
import yaml
import rosbag
import re

from tqdm import tqdm

import copy
from open3d_ros_helper import open3d_ros_helper as orh

def extractTimeAndPose(pathMsg : str):
    timeGroups = re.search(r"\bsecs: ([0-9]*).*\bnsecs: ([.0-9]*)", pathMsg, re.DOTALL).groups()    # capture secs and nsecs
    if timeGroups[1] == '':
        time = int(timeGroups[0] + '000000000')
    else:
        time = int(timeGroups[0] + timeGroups[1])
    
    TGroups = re.search(r"position:\s+x:\s+([.e0-9-]*).*y:\s+([.e0-9-]*).*z:\s+([.e0-9-]*).*orientation", pathMsg, re.DOTALL).groups()
    RGroups = re.search(r"orientation:\s+x:\s+([.e0-9-]*).*y:\s+([.e0-9-]*).*z:\s+([.e0-9-]*).*w:\s+([.e0-9-]*)", pathMsg, re.DOTALL).groups()

    T = []
    for i in TGroups:
        T += [float(i)]

    R = []
    for i in RGroups:
        R += [float(i)]

    return time, T, R   


## ---------------------- MAIN ----------------------------
def cleanPathData(bag, pathNum=1):
    """ bag = bagreader('/media/aneesh/Cherry passport vibes/RRC/CoVINS/postVINS_Feb5.bag') """

    topic='/pose_graph/path_' + str(pathNum)

    # clean retrieved data,
    count =  1
    pathTimes = []
    lastLength = 1                  # detect when to add time to the pathTimes list
    for topic, path1Poses, msgT in tqdm(bag.read_messages(topics=[topic])):
        count += 1

        path1Poses = re.sub("  - \n", ",\n", str(path1Poses))
        path1Poses = path1Poses.split(',')     # array of header: all poses (split the lsat published msg at commas)
        if(len(path1Poses) > lastLength):
            pathTimes.append(msgT)      # record each time a path message was published
            # print(count, msgT)
            lastLength += (len(path1Poses) - lastLength)


    *_, lastMSG = bag.read_messages(topics=[topic])
    extractedPath1Poses = []        # poses
    path1Poses = re.sub("  - \n", ",\n", str(lastMSG.message))
    path1Poses = path1Poses.split(',')     # array of header: all poses (split the lsat published msg at commas)

    for pathMsg in path1Poses[1:]:
        t, T, R = extractTimeAndPose(pathMsg)
        extractedPath1Poses.append([t, T, R])

    if len(extractedPath1Poses) == len(pathTimes):
        print("Times synced")
    else:
        print("Pose pcd times not synced")
        exit(0)

    # update pose times
    for i, _ in enumerate(zip(pathTimes, extractedPath1Poses)):
        extractedPath1Poses[i][0] = int(str(pathTimes[i].secs) + str(pathTimes[i].nsecs))
    
    return extractedPath1Poses

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# visualise PCDs

def visualiseBagFile(bag, extractedPath1Poses, pathNum=1, dsVoxelSize=0.1):
    """ 
    Take in extracted paths and poses from the bag file, generate a reconstructed pointcloud along with frames representing the path of the drone.
    """

    pcdTopic = '/vins_' + str(pathNum) + '/dense_pcd'

    # PCDS
    abc = 0
    latestPathIndex = 0
    reconstructedPcdPoints = []
    frames = []

    reconstruction = o3d.geometry.PointCloud()

    # iterate through each point cloud message in the bag file
    for topic, msg, t in tqdm(bag.read_messages(topics=[pcdTopic])):     
        abc += 1


        # if abc > 2300 and abc < 2500:   # exclude points recorded during initialisation
            # convert the pcd message points to 03d        
        o3dpcd = orh.rospc_to_o3dpc(msg)
        o3dpcd.paint_uniform_color([0, 0.651, 0.929])

        # pad time

        # query the closest pose from the extracted path poses
        if len(str(t.nsecs)) < 9:
            pcdTime = int(str(t.secs) + '0' * (9-len(str(t.nsecs))) + str(t.nsecs))
        else:
            pcdTime = int(str(t.secs) + str(t.nsecs))

        # check extracted poses for the latest timestamp, store the latest pose and the index of that pose
        latestPose = None
        for i in range(len(extractedPath1Poses) - 1):
            if extractedPath1Poses[i][0] <= pcdTime and extractedPath1Poses[i+1][0] > pcdTime:
                latestPose = extractedPath1Poses[i]
                break

        # error handling
        if latestPose == None:
            print("skipping")
            continue
        T = latestPose[1]

        # transform the point cloud by the queried pose, take the frame difference between o3d and ros into account
        rectify = o3dpcd.get_rotation_matrix_from_zyx((np.pi/2,-np.pi/2,np.pi))
        rollpcd = o3d.geometry.get_rotation_matrix_from_yxz((np.pi/2,-np.pi/2,0))   # align pose and pcd frames

        R = o3dpcd.get_rotation_matrix_from_quaternion(latestPose[2])     # get R and T
        o3dpcd.rotate(rollpcd, center=[0,0,0])
        o3dpcd.rotate(R, center=[0,0,0])
        o3dpcd.rotate(rectify, center=[0,0,0])
        o3dpcd.translate(T)

        # transform a coordinate frame to the pose of the drone for that pcd
        frm = o3d.geometry.TriangleMesh.create_coordinate_frame()
        frm.rotate(R, center=[0,0,0])
        frm.rotate(rectify, center=[0,0,0])
        frm.translate(T)

        # # uncomment to reduce density at the beginning 
        # aggregate pcds
        reconstruction += o3dpcd
        reconstructedPcdPoints.append(o3dpcd)
        frames.append(frm)

        # periodically downsample
        if abc % 300 == 0:
            reconstruction = reconstruction.voxel_down_sample(voxel_size=dsVoxelSize)

    return reconstruction, frames


In [3]:
bagPath = '/home/aneesh/UbuntuStorage/Airsim-Recording-Scripts/offsetSquareSine/path_n_pcd_oss.bag'
bag = rosbag.Bag(bagPath, 'r')
extractedPath1Poses = cleanPathData(bag, 1)

1135it [00:31, 35.97it/s]


Times synced


In [4]:
reconstruction1, frames1 = visualiseBagFile(bag, extractedPath1Poses, 1, 0.5)

5024it [01:00, 83.33it/s] 

skipping
skipping
skipping


In [5]:
extractedPath2Poses = cleanPathData(bag, 2)
reconstruction2, frames2 = visualiseBagFile(bag, extractedPath2Poses, 2, 0.5)
reconstruction2.paint_uniform_color([0.8, 0.651, 0])

1183it [00:48, 24.19it/s]


Times synced


4119it [00:53, 76.99it/s]


PointCloud with 2269220 points.

In [6]:
o3d.visualization.draw_geometries([reconstruction1] + frames1[::20])

In [7]:
for k in frames1[::1]:
    print(k.get_center())

[ 0.18869251 22.62948523  1.40670875]
[ 0.18869251 22.62948523  1.40670875]
[ 0.18869251 22.62948523  1.40670875]
[ 0.18869251 22.62948523  1.40670875]
[ 0.18869251 22.62948523  1.40670875]
[ 0.18869251 22.62948523  1.40670875]
[ 0.18869251 22.62948523  1.40670875]
[ 0.18869251 22.62948523  1.40670875]
[ 0.18869251 22.62948523  1.40670875]
[ 0.18869251 22.62948523  1.40670875]
[ 0.18869251 22.62948523  1.40670875]
[ 0.18869251 22.62948523  1.40670875]
[ 0.18869251 22.62948523  1.40670875]
[ 0.18869251 22.62948523  1.40670875]
[ 0.18869251 22.62948523  1.40670875]
[ 0.18869251 22.62948523  1.40670875]
[ 0.18869251 22.62948523  1.40670875]
[ 0.18869251 22.62948523  1.40670875]
[ 0.18869251 22.62948523  1.40670875]
[ 0.18869251 22.62948523  1.40670875]
[ 0.18869251 22.62948523  1.40670875]
[ 0.18869251 22.62948523  1.40670875]
[ 0.18869251 22.62948523  1.40670875]
[ 0.18869251 22.62948523  1.40670875]
[ 0.18869251 22.62948523  1.40670875]
[ 0.18869251 22.62948523  1.40670875]
[ 0.18869251

In [8]:
o3d.visualization.draw_geometries([reconstruction2] + frames2[::100])

In [9]:
# full reconstruction
o3d.visualization.draw_geometries(frames2[::50] + frames1[::50])
o3d.visualization.draw_geometries([reconstruction1, reconstruction2] + frames2[::50] + frames1[::50])

In [3]:
# # find transform between frames

# frm = o3d.geometry.TriangleMesh.create_coordinate_frame()
# frm.scale(5, [0,0,0])


# rollpcd = o3d.geometry.get_rotation_matrix_from_yxz((np.pi/2,-np.pi/2,0))
# # rollpcd = o3d.geometry.get_rotation_matrix_from_yzx((0,0,0))
# abc = copy.deepcopy(pcdt)

# # raw
# o3d.visualization.draw_geometries([frm] + [abc] + frames[::50])

# # roll
# abc.rotate(rollpcd, center=[0,0,0])
# # o3d.visualization.draw_geometries([frm] + [abc] + frames[::50])

# # bring to x axis
# # abc.rotate(o3d.geometry.get_rotation_matrix_from_yzx((np.pi/2,np.pi,0)), center=[0,0,0])
# o3d.visualization.draw_geometries([frm] + [abc] + frames[::50])


# frm2 = copy.deepcopy(frm)
# frm2.rotate(R, center=[0,0,0])
# frm2.rotate(rectify, center=[0,0,0])
# frm2.translate(T)

# abc.rotate(R, center=[0,0,0])
# abc.rotate(rectify, center=[0,0,0])
# abc.translate(T)


# o3d.visualization.draw_geometries([frm2] + [abc] + frames[::50])

In [5]:
# save the combined pcd
o3d.io.write_point_cloud("reconstruced_vins1_oss.pcd", reconstruction1 + reconstruction2)

True